<a href="https://colab.research.google.com/github/Augustus2001/Datascience/blob/main/Monkeypox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import tensorflow as tf
import os
import zipfile
from google.colab import files
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print(tf.__version__)
import matplotlib.pyplot as plt
import seaborn as sns

2.8.2


In [2]:
tf.random.set_seed(72)

In [4]:
drive.mount('/content/drive/')
zip_ref = zipfile.ZipFile("/content/drive/My Drive/monkeypox.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

Mounted at /content/drive/


In [11]:
train=os.path.join("/tmp/Fold1/Fold1/Fold1/Train")
list_train=os.listdir(train)
test=os.path.join("/tmp/Fold1/Fold1/Fold1/Test")
list_test=os.listdir(test)
len(list_train),len(list_test)

(2, 2)

In [50]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                    rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=False,
                                    )
train_ganerator=train_datagen.flow_from_directory(directory="/tmp/Fold1/Fold1/Fold1/Train",target_size=(150,150)
                                                  ,class_mode="binary"
                                                  ,batch_size=64
                                                  ,shuffle=True
)
test_datagen=ImageDataGenerator()
test_generator=test_datagen.flow_from_directory(directory="/tmp/Fold1/Fold1/Fold1/Val",
                                                target_size=(150,150),
                                                class_mode="binary",
                                                )

Found 2142 images belonging to 2 classes.
Found 420 images belonging to 2 classes.


In [40]:

model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation="relu",input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64,(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64,(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")
])

In [41]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 74, 74, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 34, 34, 64)        36928     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 17, 17, 64)      

In [42]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [43]:
class Mycallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if(logs.get("accuracy")>0.85):
      self.model.stop_training=True
callbacks=Mycallback()

In [44]:
history=model.fit(train_ganerator,epochs=40)

Epoch 1/40
34/34 [==============================] - 90s 3s/step - loss: 1.0298 - accuracy: 0.5579
Epoch 2/40
34/34 [==============================] - 87s 3s/step - loss: 0.6831 - accuracy: 0.5551
Epoch 3/40
34/34 [==============================] - 87s 3s/step - loss: 0.7198 - accuracy: 0.5570
Epoch 4/40
34/34 [==============================] - 94s 3s/step - loss: 0.6619 - accuracy: 0.6134
Epoch 5/40
34/34 [==============================] - 88s 3s/step - loss: 0.6846 - accuracy: 0.6069
Epoch 6/40
34/34 [==============================] - 92s 3s/step - loss: 0.6442 - accuracy: 0.6373
Epoch 7/40
34/34 [==============================] - 87s 3s/step - loss: 0.6463 - accuracy: 0.6485
Epoch 8/40
34/34 [==============================] - 87s 3s/step - loss: 0.6478 - accuracy: 0.6373
Epoch 9/40
34/34 [==============================] - 91s 3s/step - loss: 0.6316 - accuracy: 0.6480
Epoch 10/40
34/34 [==============================] - 93s 3s/step - loss: 0.6064 - accuracy: 0.6639
Epoch 11/40
34/34 [

In [57]:
def render_training_history(training_history):
    loss = training_history.history['loss']
    val_loss = training_history.history['val_loss']

    accuracy = training_history.history['accuracy']
    val_accuracy = training_history.history['val_accuracy']

    plt.figure(figsize=(14, 4))

    plt.subplot(1, 2, 1)
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.plot(loss, label='Training set')
    plt.plot(val_loss, label='Test set', linestyle='--')
    plt.legend()
    plt.grid(linestyle='--', linewidth=1, alpha=0.5)

    plt.subplot(1, 2, 2)
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.plot(accuracy, label='Training set')
    plt.plot(val_accuracy, label='Test set', linestyle='--')
    plt.legend()
    plt.grid(linestyle='--', linewidth=1, alpha=0.5)

    plt.show()

In [56]:
model.save("Monkeypox.h5")

In [59]:
render_training_history(history)

KeyError: ignored

In [55]:
model.evaluate(test_ganerato)

NameError: ignored